In [21]:
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from jax.nn import softmax
from jax.config import config
from pathlib import Path
from collections import namedtuple

config.update('jax_platform_name', 'cpu')
np.set_printoptions(precision=4)
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams.update({'font.size': 22})

from grl.utils import load_info
from grl.utils.mdp import get_perf
from definitions import ROOT_DIR
np.set_printoptions(precision=3)

In [22]:
results_dir = Path(ROOT_DIR, 'results', 'pomdps_mi_pi_q_abs')


In [23]:
all_results = {}

# for results_path in results_dir.iterdir():
#     if 'tmaze' not in results_path.stem:
#         continue
#     info = load_info(results_path)
results_paths = [res_path for res_path in results_dir.iterdir() if 'cheese' in res_path.stem]
results_path = results_paths[0]

In [24]:
agent_path = results_path.parent / 'agents' / f"{results_path.stem}.pkl.npy"

In [41]:
agent = load_info(agent_path)
mem = agent.memory
action_map = ['NORTH', 'SOUTH', 'EAST', 'WEST']
# info['logs']['initial_improvement_policy'], agent.memory, agent.policy
for a, m in enumerate(mem):
    print(action_map[a])
    print(mem)
    print()

NORTH
[[[[2.069e-01 7.931e-01]
   [7.746e-02 9.225e-01]]

  [[9.941e-01 5.881e-03]
   [2.174e-02 9.783e-01]]

  [[4.341e-01 5.659e-01]
   [4.677e-01 5.323e-01]]

  [[3.666e-01 6.334e-01]
   [3.534e-01 6.466e-01]]

  [[6.297e-01 3.703e-01]
   [1.838e-02 9.816e-01]]

  [[2.950e-05 1.000e+00]
   [2.568e-04 9.997e-01]]

  [[9.990e-01 9.552e-04]
   [9.996e-01 4.170e-04]]]


 [[[4.329e-03 9.957e-01]
   [4.761e-03 9.952e-01]]

  [[9.941e-01 5.926e-03]
   [2.058e-02 9.794e-01]]

  [[9.998e-01 2.430e-04]
   [9.997e-01 2.682e-04]]

  [[5.343e-04 9.995e-01]
   [1.396e-03 9.986e-01]]

  [[7.979e-02 9.202e-01]
   [9.710e-01 2.895e-02]]

  [[4.529e-01 5.471e-01]
   [7.971e-01 2.029e-01]]

  [[9.541e-01 4.591e-02]
   [9.826e-01 1.738e-02]]]


 [[[4.180e-03 9.958e-01]
   [4.188e-03 9.958e-01]]

  [[9.044e-01 9.555e-02]
   [1.974e-01 8.026e-01]]

  [[8.760e-01 1.240e-01]
   [9.623e-01 3.770e-02]]

  [[2.493e-01 7.507e-01]
   [4.760e-01 5.240e-01]]

  [[9.941e-01 5.903e-03]
   [9.582e-04 9.990e-01]]

  

In [43]:
tol = 0.1

SET = np.array([
    [0, 1], 
    [0, 1]
])

RESET = np.array([
    [1, 0], 
    [1, 0]
])

HOLD = np.array([
    [1, 0], 
    [0, 1]
])

FLIP = np.array([
    [0, 1],
    [1, 0]
])

for a, act_mem in enumerate(agent.memory):
    for obs_idx, obs_act_mem in enumerate(act_mem):
        if obs_idx == act_mem.shape[0] - 1:
            continue
            
        if np.allclose(obs_act_mem, SET, atol=tol):
            print(f"(obs: {obs_idx}, action: {action_map[a]}) -> SET")
        elif np.allclose(obs_act_mem, RESET, atol=tol):
            print(f"(obs: {obs_idx}, action: {action_map[a]}) -> RESET")
        elif np.allclose(obs_act_mem, HOLD, atol=tol):
            print(f"(obs: {obs_idx}, action: {action_map[a]}) -> HOLD")
        elif np.allclose(obs_act_mem, FLIP, atol=tol):
            print(f"(obs: {obs_idx}, action: {action_map[a]}) -> FLIP")
        else:
            continue


(obs: 1, action: NORTH) -> HOLD
(obs: 5, action: NORTH) -> SET
(obs: 0, action: SOUTH) -> SET
(obs: 1, action: SOUTH) -> HOLD
(obs: 2, action: SOUTH) -> RESET
(obs: 3, action: SOUTH) -> SET
(obs: 4, action: SOUTH) -> FLIP
(obs: 0, action: EAST) -> SET
(obs: 4, action: EAST) -> HOLD
(obs: 3, action: WEST) -> RESET
(obs: 4, action: WEST) -> HOLD


In [35]:
agent.memory.shape

(4, 7, 2, 2)